In [ ]:
#https://arxiv.org/pdf/1404.2188.pdf
#A Convolutional Neural Network for Modelling Sentences


sentences를 다룰 때, cnn을 사용하면 거리가 먼 words 끼리의 sentences가 pooling 될 일이없음.
하지만 k-max pooling을 통해, k개의 active한 값을 뽑아내면서 기존 pooling의 한계 극복

i번째 layer의 k = max(ktop, (L-i)*s/L) #L 네트워크 깊이, 문장길이 s, ktop 최상단 층 convolutional layer




In [1]:
import glob
import pandas as pd
import re
import os
import numpy as np

In [3]:
glob.glob("../../../downloads/glove.6B/*")

['../../../downloads/glove.6B\\glove.6B.100d.txt',
 '../../../downloads/glove.6B\\glove.6B.200d.txt',
 '../../../downloads/glove.6B\\glove.6B.300d.txt',
 '../../../downloads/glove.6B\\glove.6B.50d.txt']

In [4]:
raw_data = pd.read_csv("../../../downloads/toxic/train.csv")

In [5]:
raw_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
raw_data['comment_text'] = raw_data['comment_text'].apply(lambda x : x.replace("\n"," "))

In [7]:
raw_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" More I can't make any real suggestions on im...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [64]:
labels = raw_data.iloc[:,2:]

In [8]:
raw_data.shape

(159571, 8)

In [11]:
texts = ' '.join(raw_data.iloc[:10000,:]['comment_text'].values)

In [12]:
p = re.compile(r"[\(\[].*?[\)\]]")
pp = re.compile("[^a-zA-Z ]")


In [13]:
texts = re.sub(p,"",texts)
texts = re.sub(pp,"",texts)

In [14]:
texts = texts.replace("  "," ").replace("  "," ").replace("  "," ")

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

C:\Users\ie-02\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [38]:
maxlen = 100
max_words = 10000

#texts가 존재해야함.


tokenizer = Tokenizer()
tokenizer.fit_on_texts([texts])
tokenizer.num_words = max_words
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(raw_data['comment_text'][:10000].values)


data = pad_sequences(sequences, maxlen = maxlen)



In [46]:
embeddings_index = {}
glove_dir = "../../../downloads/glove.6B/glove.6B.300d.txt"
with open(os.path.join(glove_dir),encoding='utf-8') as fp:
    for line in fp:
        values = line.split()
        word = values[0]
        params = np.asarray(values[1:],dtype = 'float32')
        embeddings_index[word] = params


In [33]:
len(embeddings_index.keys())

400000

In [102]:
len(word_index)

36593

In [57]:
embedding_dim = 300

matrix_size = min(len(word_index),max_words)

embedding_matrix = np.zeros((len(word_index)+1,embedding_dim))

for word, i in word_index.items(): #
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and i <= matrix_size:
        embedding_matrix[i] = embedding_vector

In [60]:
from keras.layers import Embedding, Flatten,Dense,Input,Conv1D,Flatten,MaxPooling1D,Concatenate
from keras.models import Model

In [51]:
input_layer = Input(shape = (maxlen,))

In [58]:
embedding_layer = Embedding(len(word_index) + 1,embedding_dim,weights =[embedding_matrix],\
                           input_length = maxlen, trainable = False)(input_layer)

In [65]:
x1 = Conv1D(128,1, activation = 'relu')(embedding_layer)
x2 = Conv1D(128,2, activation = 'relu')(embedding_layer)
x3 = Conv1D(128,3, activation = 'relu')(embedding_layer)
x4 = Conv1D(128,4, activation = 'relu')(embedding_layer)
x5 = Conv1D(128,5, activation = 'relu')(embedding_layer)

x1 = MaxPooling1D(2)(x1)
x2 = MaxPooling1D(2)(x2)
x3 = MaxPooling1D(2)(x3)
x4 = MaxPooling1D(2)(x4)
x5 = MaxPooling1D(2)(x5)

x1 = Conv1D(128,1,activation = 'relu')(x1)
x2 = Conv1D(128,2,activation = 'relu')(x2)
x3 = Conv1D(128,3,activation = 'relu')(x3)
x4 = Conv1D(128,4,activation = 'relu')(x4)
x5 = Conv1D(128,5,activation = 'relu')(x5)

x1 = MaxPooling1D(2)(x1)
x2 = MaxPooling1D(2)(x2)
x3 = MaxPooling1D(2)(x3)
x4 = MaxPooling1D(2)(x4)
x5 = MaxPooling1D(2)(x5)

x1 = Conv1D(128,1,activation = 'relu')(x1)
x2 = Conv1D(128,2,activation = 'relu')(x2)
x3 = Conv1D(128,3,activation = 'relu')(x3)
x4 = Conv1D(128,4,activation = 'relu')(x4)
x5 = Conv1D(128,5,activation = 'relu')(x5)

x1 = Flatten()(x1)
x2 = Flatten()(x2)
x3 = Flatten()(x3)
x4 = Flatten()(x4)
x5 = Flatten()(x5)

x = Concatenate()([x1,x2,x3,x4,x5])

x = Dense(128,activation = 'relu')(x)
x = Dense(len(labels.columns),activation = 'softmax')(x)

model = Model(input_layer, x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [76]:
l_test_x = list(map(lambda x : np.array(x),sequences))

In [83]:
l_test_x = np.array(data)

In [85]:
l_test_x.shape

(10000, 100)

In [ ]:
model.fit(l_test_x,labels.iloc[:10000])

Epoch 1/1
 6528/10000 [==================>...........] - ETA: 41s - loss: 0.3292 - acc: 0.8644